# Libraries

In [2]:
import os
import requests
import json

# Get token programmmatically

In [ ]:

def oAuth_example():

  url = "https://oauth2.bitquery.io/oauth2/token"

  payload = 'grant_type=client_credentials&client_id=b5d0cf60-576d-4edc-a04f-779f6022470a&client_secret=e1WJBc6tYEPD9Z2CEo~1hektZy&scope=api'

  headers = {'Content-Type': 'application/x-www-form-urlencoded'}

  response = requests.request("POST", url, headers=headers, data=payload)
  resp = json.loads(response.text)
  print(resp)
  access_token=resp['access_token']

  url_graphql = "https://streaming.bitquery.io/graphql"
  headers_graphql = {
      'Content-Type': 'application/json',
      'Authorization': f'Bearer {access_token}'
  }



oAuth_example()

{'access_token': 'ory_at_jgssi6zitf7IgoG1wbccsZYAUWp5kqi8NFUVAHQlQtI.HKttVtk7Rf_NQt3aQiHEGMcx2d-VpIRj6vNZnQmiceg', 'expires_in': 31553999, 'scope': 'api', 'token_type': 'bearer'}


In [3]:
API_KEY = os.getenv('API_KEY')

In [4]:
API_KEY

'Bearer ory_at_bsdgYItUpSmMah9ahPhmDZYLE2xSDwnwOpeFye2Vigo.kt8gzV0EgDsaxmFz8Z9RQEs_oejv7fenu6AwdBtl5mI'

In [18]:
from datetime import date
today = date.today()

In [19]:
today

datetime.date(2025, 2, 6)

In [26]:
query = """{
  EVM(dataset: combined, network: arbitrum) {
    Transfers(
      where: {
        Transaction: {
          To: { is: "0xe35e9842fceaca96570b734083f4a58e8f7c5f2a" }
        }
        Block: { Date: { is: "2024-12-12" } }
      }
      limit: { count: 10 }
      orderBy: { descending: Block_Number }
    ) {
      Transfer {
        Amount
        Currency {
          Name
          Symbol
          SmartContract
        }
        Sender
        Receiver
      }
      Call {
        Signature {
          Name
        }
        From
        To
        Value
        CallPath
      }
      Block {
        Number
        Time
      }
    }
  }
}
"""
url = "https://streaming.bitquery.io/graphql"

payload = json.dumps({
   "query": query,

   "variables": "{}"
})
headers = {
   'Content-Type': 'application/json',
   'Authorization': API_KEY
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"data":{"EVM":{"Transfers":[{"Block":{"Number":"284143373","Time":"2024-12-12T23:59:30Z"},"Call":{"CallPath":[0,2],"From":"0xe35e9842fceaca96570b734083f4a58e8f7c5f2a","Signature":{"Name":""},"To":"0xb2d26066c4a141784ef08d063d09f5b25546d8d3","Value":"9061914606197072"},"Transfer":{"Amount":"0.009061914606197072","Currency":{"Name":"Ethereum","SmartContract":"0x","Symbol":"ETH"},"Receiver":"0xb2d26066c4a141784ef08d063d09f5b25546d8d3","Sender":"0xe35e9842fceaca96570b734083f4a58e8f7c5f2a"}},{"Block":{"Number":"284143373","Time":"2024-12-12T23:59:30Z"},"Call":{"CallPath":[0,0,0],"From":"0x82af49447d8a07e3bd95bd0d56f35241523fbab1","Signature":{"Name":"transferFrom"},"To":"0x8b194beae1d3e0788a1a35173978001acdfba668","Value":"0"},"Transfer":{"Amount":"0.009061914606197072","Currency":{"Name":"Wrapped Ether","SmartContract":"0x82af49447d8a07e3bd95bd0d56f35241523fbab1","Symbol":"WETH"},"Receiver":"0xe35e9842fceaca96570b734083f4a58e8f7c5f2a","Sender":"0x3d7dc36aa2b542ad239012730dfdb23f03d75be9"}

In [6]:
import requests


def run_query(query, api_key):  # A simple function to use requests.post to make the API call.
    headers = {'X-API-KEY': api_key}
    request = requests.post(url,
                            json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed and return code is {}.      {}'.format(request.status_code,
                        request.text))

In [7]:

# The GraphQL query

query = """
    query{
      bitcoin{
        blocks{
          count
        }
      }
    }
"""
result = run_query(query, api_key=API_KEY)  # Execute the query
print ('Result - {}'.format(result))

Exception: Query failed and return code is 401.      Unauthorized. You have to use Authorization or X-API-KEY tokens as described in the documentation https://docs.bitquery.io/docs/category/authorization

In [14]:
import websocket
import json

BITQUERY_API_KEY = API_KEY  # Replace with your Bitquery V2 API key

def on_message(ws, message):
    data = json.loads(message)
    print("New Transaction:")
    print(json.dumps(data, indent=4))  # Pretty-print the JSON response

def on_error(ws, error):
    print("Error:", error)

def on_close(ws, close_status_code, close_msg):
    print("Closed:", close_status_code, close_msg)

def on_open(ws):
    print("Connected to Bitquery WebSocket")
    query = {
        "type": "subscribe",
        "query": {
            "network": "ethereum",  # Change to "bsc", "polygon", etc.
            "limit": 10  # Adjust the limit of transactions to receive at a time
        }
    }
    ws.send(json.dumps(query))  # Send subscription request

ws = websocket(
    "wss://streaming.bitquery.io/trades",
    header={"X-API-KEY": BITQUERY_API_KEY},
    on_message=on_message,
    on_error=on_error,
    on_close=on_close
)

ws.on_open = on_open
ws.run_forever()


TypeError: 'module' object is not callable

/tmp/ipykernel_86390/2896509904.py:27: RuntimeWarning: coroutine 'coinbase_ws' was never awaited
  asyncio.get_event_loop()(coinbase_ws())


TypeError: '_UnixSelectorEventLoop' object is not callable

In [ ]:
import asyncio
import websockets
import json
import nest_asyncio

nest_asyncio.apply()  # Fix for Jupyter Notebook

async def coinbase_ws():
    url = "wss://ws-feed.exchange.coinbase.com"

    # Subscription message to get real-time BTC-USD price updates
    subscribe_msg = {
        "type": "subscribe",
        "channels": [{"name": "ticker", "product_ids": ["BTC-USD"]}]
    }

    async with websockets.connect(url) as ws:
        await ws.send(json.dumps(subscribe_msg))
        print("✅ Connected to Coinbase WebSocket API...")

        while True:
            response = await ws.recv()
            data = json.loads(response)

            # Print only valid price updates
            if "price" in data:
                print(f"🔹 BTC-USD Price: ${data}")

# Run event loop safely
asyncio.get_event_loop().run_until_complete(coinbase_ws())


✅ Connected to Coinbase WebSocket API...


In [21]:
import requests

COINBASE_REST_URL = "https://api.exchange.coinbase.com/products/BTC-USD/ticker"

def get_btc_price():
    response = requests.get(COINBASE_REST_URL)
    if response.status_code == 200:
        data = response.json()
        print(data)
        return data.get("price")  # Extract and return the price
    else:
        return None  # Handle errors gracefully

btc_price = get_btc_price()
print(f"BTC-USD Price: ${btc_price}")


{'ask': '96147.71', 'bid': '96140.07', 'volume': '10929.36575755', 'trade_id': 777161332, 'price': '96147.89', 'size': '0.00035696', 'time': '2025-02-08T12:11:08.404859Z', 'rfq_volume': '42.218618'}
BTC-USD Price: $96147.89


In [22]:
import requests

COINBASE_STATS_URL = "https://api.exchange.coinbase.com/products/BTC-USD/stats"

def get_btc_market_data():
    response = requests.get(COINBASE_STATS_URL)
    if response.status_code == 200:
        data = response.json()
        return {
            "price": data.get("last"),   # Latest BTC price
            "high": data.get("high"),    # 24-hour high price
            "low": data.get("low"),      # 24-hour low price
            "volume": data.get("volume") # 24-hour traded volume
        }
    return None  # Handle errors

btc_data = get_btc_market_data()
if btc_data:
    print(f"🔹 BTC-USD Price: ${btc_data['price']}")
    print(f"📈 High (24h): ${btc_data['high']}")
    print(f"📉 Low (24h): ${btc_data['low']}")
    print(f"📊 Volume (24h): {btc_data['volume']} BTC")
else:
    print("⚠️ Failed to fetch market data.")


🔹 BTC-USD Price: $96127.05
📈 High (24h): $100235.79
📉 Low (24h): $95614.96
📊 Volume (24h): 10934.07888009 BTC


In [10]:
import requests

In [6]:
url = "wss://ws-feed.exchange.coinbase.com"
response = requests.get(url)
if response.status_code == 200:
    response = response.json()

InvalidSchema: No connection adapters were found for 'wss://ws-feed.exchange.coinbase.com'

In [1]:
import websocket
import json

COINBASE_API_URL = "wss://ws-feed.exchange.coinbase.com"

def coinbase_ws():
    url = COINBASE_API_URL

    # Subscription message to get real-time BTC-USD price updates
    subscribe_msg = {
        "type": "subscribe",
        "channels": [{"name": "ticker", "product_ids": ["BTC-USD"]}]
    }

    # Create a WebSocket connection
    ws = websocket.create_connection(url)

    # Send subscription message
    ws.send(json.dumps(subscribe_msg))

    # Receive one message
    response = ws.recv()
    data = json.loads(response)

    ws.close()  # Close connection after receiving one message

    # Return price if available
    return data.get("price")

btc_price = coinbase_ws()
print(f"Returned BTC Price: ${btc_price}")


AttributeError: module 'websocket' has no attribute 'create_connection'

In [23]:
import time

while True:
    btc_data = get_btc_market_data()
    if btc_data:
        print(f"🔹 BTC-USD Price: ${btc_data['price']}")
        print(f"📈 High (24h): ${btc_data['high']}")
        print(f"📉 Low (24h): ${btc_data['low']}")
        print(f"📊 Volume (24h): {btc_data['volume']} BTC")
    else:
        print("⚠️ Failed to fetch market data.")
    
    time.sleep(10)  # Wait 10 seconds before updating again


🔹 BTC-USD Price: $96096.99
📈 High (24h): $100235.79
📉 Low (24h): $95614.96
📊 Volume (24h): 10934.54487764 BTC
🔹 BTC-USD Price: $96101.98
📈 High (24h): $100235.79
📉 Low (24h): $95614.96
📊 Volume (24h): 10934.5742573 BTC
🔹 BTC-USD Price: $96089.06
📈 High (24h): $100235.79
📉 Low (24h): $95614.96
📊 Volume (24h): 10934.61588982 BTC
🔹 BTC-USD Price: $96063.2
📈 High (24h): $100235.79
📉 Low (24h): $95614.96
📊 Volume (24h): 10934.83726602 BTC
🔹 BTC-USD Price: $96063.2
📈 High (24h): $100235.79
📉 Low (24h): $95614.96
📊 Volume (24h): 10934.83726602 BTC


KeyboardInterrupt: 

In [26]:
import websocket
import json
import time
from datetime import datetime

COINBASE_API_URL = "wss://ws-feed.exchange.coinbase.com"

# Store the latest trade data
latest_data = {
    "time": None,
    "price": None,
    "high": None,
    "low": None,
    "open": None,
    "close": None,
    "volume": 0
}

# Function to handle WebSocket messages
def on_message(ws, message):
    global latest_data

    data = json.loads(message)
    if "price" in data:
        current_time = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
        latest_data["time"] = current_time
        latest_data["price"] = float(data["price"])
        # You can store high, low, and volume by tracking price changes over time
        print(f"Time: {latest_data['time']} | Price: {latest_data['price']}")

    if "side" in data:  # Volume tracking logic
        latest_data["volume"] += float(data["size"])

# Function to subscribe to the WebSocket
def coinbase_ws():
    ws = websocket.WebSocketApp(COINBASE_API_URL, on_message=on_message)

    # Subscription message for BTC-USD ticker updates
    subscribe_msg = {
        "type": "subscribe",
        "channels": [{"name": "ticker", "product_ids": ["BTC-USD"]}]
    }

    ws.on_open = lambda ws: ws.send(json.dumps(subscribe_msg))
    ws.run_forever()

# Start the WebSocket
coinbase_ws()


AttributeError: module 'websocket' has no attribute 'WebSocketApp'

In [4]:
import asyncio
import websockets
import json

# WebSocket URL
COINBASE_API_URL = "wss://ws-feed.exchange.coinbase.com"

# Function to handle incoming messages
async def handle_messages():
    # Define the WebSocket connection
    async with websockets.connect(COINBASE_API_URL) as ws:
        # Prepare the subscription message
        subscribe_msg = {
            "type": "subscribe",
            "channels": [
                {"name": "ticker", "product_ids": ["BTC-USD"]},   # Price (Ticker) updates
                {"name": "level2", "product_ids": ["BTC-USD"]},    # Order book (Level 2) updates
            ]
        }
        
        # Send the subscription request
        await ws.send(json.dumps(subscribe_msg))
        print("✅ Connected to Coinbase WebSocket API...")

        while True:
            # Wait for new messages from the WebSocket connection
            response = await ws.recv()
            data = json.loads(response)
            
            # Check and process the 'ticker' data (e.g., price updates)
            if data.get("type") == "ticker":
                print("🔹 Price Update:")
                print(f"Price: {data['price']}")
                print(f"Time: {data['time']}")
                print(f"Best Bid: {data['best_bid']}")
                print(f"Best Ask: {data['best_ask']}")
                print(f"Volume: {data['volume_24h']}")
                # print(f"Change (24h): {data['change_24h']}")
            
            # Check and process the 'level2' data (e.g., order book updates)
            elif data.get("type") == "l2update":
                print("📉 Order Book Update:")
                print("Changes in order book:", data['changes'])
            
            # Other possible event types (trade, etc.)
            else:
                print("🔄 Other Data Event:", data)

# Running the event loop for the WebSocket connection
asyncio.run(handle_messages())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [5]:
import asyncio
import websockets
import json

COINBASE_WS_URL = "wss://ws-feed.exchange.coinbase.com"

async def fetch_websocket_data():
    async with websockets.connect(COINBASE_WS_URL) as ws:
        subscribe_msg = {
            "type": "subscribe",
            "channels": [{"name": "ticker", "product_ids": ["BTC-USD"]}]
        }
        await ws.send(json.dumps(subscribe_msg))

        while True:
            response = await ws.recv()
            data = json.loads(response)
            print("Real-Time Data:", data)

# Run the WebSocket client
asyncio.run(fetch_websocket_data())


RuntimeError: asyncio.run() cannot be called from a running event loop

In [6]:
import websocket
import json

COINBASE_WS_URL = "wss://ws-feed.exchange.coinbase.com"

def on_message(ws, message):
    data = json.loads(message)
    print("Real-Time Data:", data)

def on_open(ws):
    subscribe_msg = {
        "type": "subscribe",
        "channels": [{"name": "ticker", "product_ids": ["BTC-USD"]}]
    }
    ws.send(json.dumps(subscribe_msg))

# Create WebSocket connection
ws = websocket.WebSocketApp(COINBASE_WS_URL, on_message=on_message, on_open=on_open)
ws.run_forever()


AttributeError: module 'websocket' has no attribute 'WebSocketApp'

In [2]:
import websocket
import threading
import json
import time

COINBASE_WS_URL = "wss://ws-feed.exchange.coinbase.com"

def on_message(ws, message):
    data = json.loads(message)
    print("Real-Time Data:", data)

def on_open(ws):
    subscribe_msg = {
        "type": "subscribe",
        "channels": [{"name": "ticker", "product_ids": ["BTC-USD"]}]
    }
    ws.send(json.dumps(subscribe_msg))

# Run WebSocket in a separate thread
ws = websocket.WebSocketApp(COINBASE_WS_URL, on_message=on_message, on_open=on_open)
wst = threading.Thread(target=ws.run_forever)
wst.start()

# Do other tasks while WebSocket runs
for i in range(10):
    print("Running other tasks...")
    time.sleep(1)


Running other tasks...
Running other tasks...
Running other tasks...
Real-Time Data: {'type': 'subscriptions', 'channels': [{'name': 'ticker', 'product_ids': ['BTC-USD'], 'account_ids': None}]}
Real-Time Data: {'type': 'ticker', 'sequence': 98835760224, 'product_id': 'BTC-USD', 'price': '96062', 'open_24h': '99792.21', 'volume_24h': '8762.65729714', 'low_24h': '95614.96', 'high_24h': '99792.21', 'volume_30d': '379616.05055367', 'best_bid': '96052.03', 'best_bid_size': '0.00014255', 'best_ask': '96062.00', 'best_ask_size': '0.05154146', 'side': 'buy', 'time': '2025-02-08T14:23:58.921101Z', 'trade_id': 777190643, 'last_size': '0.00045854'}
Real-Time Data: {'type': 'ticker', 'sequence': 98835761148, 'product_id': 'BTC-USD', 'price': '96061.97', 'open_24h': '99792.21', 'volume_24h': '8762.65734699', 'low_24h': '95614.96', 'high_24h': '99792.21', 'volume_30d': '379616.05060352', 'best_bid': '96052.86', 'best_bid_size': '0.00014255', 'best_ask': '96061.98', 'best_ask_size': '0.00000827', 'si

Real-Time Data: {'type': 'ticker', 'sequence': 98835764254, 'product_id': 'BTC-USD', 'price': '96062.71', 'open_24h': '99792.21', 'volume_24h': '8762.66760095', 'low_24h': '95614.96', 'high_24h': '99792.21', 'volume_30d': '379616.06085748', 'best_bid': '96056.41', 'best_bid_size': '0.00014255', 'best_ask': '96062.71', 'best_ask_size': '0.02094189', 'side': 'buy', 'time': '2025-02-08T14:24:08.343174Z', 'trade_id': 777190649, 'last_size': '0.00505811'}
Real-Time Data: {'type': 'ticker', 'sequence': 98835764433, 'product_id': 'BTC-USD', 'price': '96062.65', 'open_24h': '99792.21', 'volume_24h': '8762.66794116', 'low_24h': '95614.96', 'high_24h': '99792.21', 'volume_30d': '379616.06119769', 'best_bid': '96057.36', 'best_bid_size': '0.00014255', 'best_ask': '96062.65', 'best_ask_size': '0.02568447', 'side': 'buy', 'time': '2025-02-08T14:24:08.737216Z', 'trade_id': 777190650, 'last_size': '0.00034021'}
Real-Time Data: {'type': 'ticker', 'sequence': 98835764835, 'product_id': 'BTC-USD', 'pric

In [5]:
import websocket
import threading
import json
from queue import Queue

COINBASE_WS_URL = "wss://ws-feed.exchange.coinbase.com"

# Thread-safe queue to store messages
data_queue = Queue()

def on_message(ws, message):
    """Callback for handling incoming WebSocket messages."""
    data = json.loads(message)
    
    # Filter out non-market data (subscription confirmations, heartbeats, etc.)
    if "type" in data and data["type"] == "ticker":
        data_queue.put(data)  # Store only market data in the queue

def on_open(ws):
    """Callback when WebSocket opens, sends subscription message."""
    subscribe_msg = {
        "type": "subscribe",
        "channels": [{"name": "ticker", "product_ids": ["BTC-USD"]}]
    }
    ws.send(json.dumps(subscribe_msg))

def run_websocket():
    """Starts the WebSocket connection in a separate thread."""
    ws = websocket.WebSocketApp(COINBASE_WS_URL, on_message=on_message, on_open=on_open)
    ws.run_forever()

# Start WebSocket in a background thread
ws_thread = threading.Thread(target=run_websocket, daemon=True)
ws_thread.start()

def get_real_time_data():
    """Generator function to yield real-time data."""
    while True:
        data = data_queue.get()  # Retrieve filtered market data from queue
        yield data

# Example Usage: Fetch only market data
for msg in get_real_time_data():
    print("Filtered Market Data:", msg)


Filtered Market Data: {'type': 'ticker', 'sequence': 98836053092, 'product_id': 'BTC-USD', 'price': '96063.89', 'open_24h': '99792.21', 'volume_24h': '8772.95187919', 'low_24h': '95614.96', 'high_24h': '99792.21', 'volume_30d': '379626.34513572', 'best_bid': '96060.43', 'best_bid_size': '0.06480000', 'best_ask': '96063.89', 'best_ask_size': '0.07935526', 'side': 'buy', 'time': '2025-02-08T14:33:14.985341Z', 'trade_id': 777192907, 'last_size': '0.00064474'}
Filtered Market Data: {'type': 'ticker', 'sequence': 98836053634, 'product_id': 'BTC-USD', 'price': '96062.13', 'open_24h': '99792.21', 'volume_24h': '8772.95198622', 'low_24h': '95614.96', 'high_24h': '99792.21', 'volume_30d': '379626.34524275', 'best_bid': '96062.12', 'best_bid_size': '0.00277372', 'best_ask': '96062.13', 'best_ask_size': '0.00008905', 'side': 'buy', 'time': '2025-02-08T14:33:16.022104Z', 'trade_id': 777192908, 'last_size': '0.00010703'}
Filtered Market Data: {'type': 'ticker', 'sequence': 98836053636, 'product_id'

KeyboardInterrupt: 